In [1]:
import sys
sys.path.append("../mypkg")

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from easydict import EasyDict as edict
from tqdm import trange

plt.style.use('ggplot')
plt.rcParams["savefig.bbox"] = "tight"

In [11]:
from splines import obt_bsp_basis_Rfn
from models.base_model import BaseModel
from models.logistic_model import LogisticModel
from utils.matrix import col_vec2mat_fn, col_vec_fn, vecidx2matidx

In [4]:
torch.set_default_tensor_type(torch.DoubleTensor)

## Parameters

In [5]:
paras = edict()
paras.npts = 20 # num of pts to evaluate X(s)
paras.d = 10 # num of ROIs
paras.q = 5 # num of other covariates

paras.bsp = edict()
paras.bsp.aknots_raw = np.linspace(0, 1, 20)
paras.bsp.iknots = paras.bsp.aknots_raw[1:-1]
paras.bsp.bknots = np.array([0, 1])
paras.bsp.ord = 4
paras.bsp.x = np.linspace(0, 1, paras.npts+1)[:-1]
paras.bsp.basis_mat = obt_bsp_basis_Rfn(paras.bsp.x, 
                                        paras.bsp.iknots, 
                                        paras.bsp.bknots, 
                                        paras.bsp.ord)
paras.bsp.N = paras.bsp.basis_mat.shape[1] # num of basis for bsp

In [6]:
M = 30
X = torch.randn(M, paras.d, paras.npts) # M x d x npts
Z = torch.randn(M, paras.q) # M x q
Y = torch.randn(M)
Y = (Y >0.5).float()

alp = torch.randn(paras.q)
Gam = torch.randn(paras.bsp.N, paras.d) # N x d
basis_mat = torch.DoubleTensor(paras.bsp.basis_mat) # npts x N


In [7]:
model = LogisticModel(Y, Z, X, basis_mat)

### Check first dev

In [8]:
dlt = 1e-8 

num_alp = torch.zeros_like(alp)
for ix in range(len(alp)):
    alp_c = alp.clone()
    alp_c[ix] = alp[ix] + dlt

    v1 = model.log_lik(alp, Gam)
    v2 = model.log_lik(alp_c, Gam)
    num_alp[ix] = (v2-v1)/dlt

num_gam = torch.zeros_like(Gam)
for ix in range(Gam.shape[0]):
    for iy in range(Gam.shape[1]):
        Gam_c = Gam.clone()
        Gam_c[ix, iy] = Gam[ix, iy] + dlt*np.sqrt(paras.bsp.N)
        
        v1 = model.log_lik(alp, Gam)
        v2 = model.log_lik(alp, Gam_c)
        num_gam[ix, iy] = (v2-v1)/dlt

In [12]:
grad1 = model.log_lik_der1(alp, Gam)
num_grad = torch.cat([num_alp, col_vec_fn(num_gam)])

In [13]:
diff = grad1 - num_grad
torch.norm(diff)/torch.norm(grad1)

tensor(4.6307e-07)

### Check second dev

In [14]:
dlt = 1e-6

num_the_len = paras.q + paras.d*paras.bsp.N
num_grad2 = torch.zeros(num_the_len, num_the_len)

vraws = model.log_lik_der1(alp, Gam)
for ix in trange(num_the_len):
    alpx = alp.clone()
    Gamx = Gam.clone()
        
    # alp part
    if ix <= (paras.q-1):
        dltx = dlt
        alpx[ix] = alp[ix] + dltx
    else:
        dltx = dlt*np.sqrt(paras.bsp.N)
        loc1, loc2 = vecidx2matidx(ix-paras.q, nrow=paras.bsp.N)
        Gamx[loc1, loc2] = Gam[loc1, loc2] + dltx
            
    v1s = model.log_lik_der1(alpx, Gamx)
    der2_vs = (v1s-vraws)/dlt
        
    #num_grad2[ix, :] = der2_vs
    num_grad2[ix, ix:] = der2_vs[ix:]
num_grad2 = num_grad2 + num_grad2.T - torch.diag(torch.diag(num_grad2))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:00<00:00, 5446.61it/s]


In [15]:
grad2 = model.log_lik_der2(alp, Gam)
diff = grad2 - num_grad2
torch.norm(diff)/torch.norm(grad2)

tensor(1.8350e-07)

In [16]:
mat = (diff.abs()/grad2.abs())
mat.max()

tensor(0.0247)

In [17]:
grad2.abs().min()

tensor(3.2986e-13)

In [18]:
grad2[mat==mat.max()]

tensor([-3.4706e-10, -3.4706e-10])

In [19]:
diff[mat==mat.max()]

tensor([8.5581e-12, 8.5581e-12])